In [2]:
# Install libs
import sys
!{sys.executable} -m pip install paramiko

In [26]:
# Read access details 
import json
with open('config.json') as f:
    data = json.load(f)


In [27]:
#Connect to server ps: make sure VPN is on 
import paramiko
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(data['host'], username=data['username'], password=data['password'])


In [ ]:
#Better Method 
sftp_client = client.open_sftp()
remote_file = sftp_client.open('./wikidump/enwiki-20190201-pages-meta-history27.xml-p53988643p54279382','rb')
try:
    for line in remote_file:
        line = line.decode("utf8", "ignore").strip()
        if line == "<page>":
            page_text = ""
            page_text += "\n" + line
            if line == "</page>":
                revisions_list = []
                page_tree = bs4.BeautifulSoup(page_text, "html.parser")
                previous_text = ""
                for revision_tag in page_tree.find_all("revision"):
                    revision_text = revision_tag.find_all("text")[0].text
                    if previous_text:
                        a = [t for t in self._wikitext_segmenter(previous_text) if t]
                        b = [t for t in self._wikitext_segmenter(revision_text) if t]
                        s = difflib.SequenceMatcher(None, a, b)
                        for tag, i1, i2, j1, j2 in s.get_opcodes():
                            if tag == "equal":
                                continue
                            revisions_list.append({
                                "old_value": a[i1:i2],
                                "new_value": b[j1:j2],
                                "left_context": a[i1 - self.REVISION_WINDOW_SIZE:i1],
                                "right_context": a[i2:i2 + self.REVISION_WINDOW_SIZE]
                            })
                    previous_text = revision_text
                if revisions_list:
                    page_counter += 1
                    if self.VERBOSE and page_counter % 100 == 0:
                        for entry in revisions_list:
                            print("----------Page Counter:---------\n", page_counter,
                                  "\n----------Old Value:---------\n", entry["old_value"],
                                  "\n----------New Value:---------\n", entry["new_value"],
                                  "\n----------Left Context:---------\n", entry["left_context"],
                                  "\n----------Right Context:---------\n", entry["right_context"],
                                  "\n==============================")
                    json.dump(revisions_list, open(os.path.join(rdd_folder_path, page_tree.id.text + ".json"), "w"))
finally:
    remote_file.close()

In [ ]:
    def _wikitext_segmenter(wikitext):
        """
        This method takes a Wikipedia page revision text in wikitext and segments it recursively.
        """
        def recursive_segmenter(node):
            if isinstance(node, str):
                segments_list.append(node)
            # elif isinstance(node, mwparserfromhell.nodes.text.Text):
            #     segments_list.append(node.value)
            elif not node:
                pass
            elif isinstance(node, mwparserfromhell.wikicode.Wikicode):
                for n in node.nodes:
                    if isinstance(n, str):
                        recursive_segmenter(n)
                    elif isinstance(n, mwparserfromhell.nodes.text.Text):
                        recursive_segmenter(n.value)
                    elif isinstance(n, mwparserfromhell.nodes.heading.Heading):
                        recursive_segmenter(n.title)
                    elif isinstance(n, mwparserfromhell.nodes.tag.Tag):
                        recursive_segmenter(n.contents)
                    elif isinstance(n, mwparserfromhell.nodes.wikilink.Wikilink):
                        if n.text:
                            recursive_segmenter(n.text)
                        else:
                            recursive_segmenter(n.title)
                    elif isinstance(n, mwparserfromhell.nodes.external_link.ExternalLink):
                        # recursive_parser(n.url)
                        recursive_segmenter(n.title)
                    elif isinstance(n, mwparserfromhell.nodes.template.Template):
                        recursive_segmenter(n.name)
                        for p in n.params:
                            # recursive_parser(p.name)
                            recursive_segmenter(p.value)
                    elif isinstance(n, mwparserfromhell.nodes.html_entity.HTMLEntity):
                        segments_list.append(n.normalize())
                    elif not n or isinstance(n, mwparserfromhell.nodes.comment.Comment) or \
                            isinstance(n, mwparserfromhell.nodes.argument.Argument):
                        pass
                    else:
                        sys.stderr.write("Inner layer unknown node found: {}, {}\n".format(type(n), n))
            else:
                sys.stderr.write("Outer layer unknown node found: {}, {}\n".format(type(node), node))

        segments_list = []
        parsed_wikitext = mwparserfromhell.parse(wikitext)
        recursive_segmenter(parsed_wikitext)
        return segments_list

In [ ]:
   def extract_revisions(self, wikipedia_dumps_folder):
        """
        This method takes the folder path of Wikipedia page revision history dumps and extracts the value-based corrections.
        """
        rd_folder_path = os.path.join(wikipedia_dumps_folder, "revision-data")
        if not os.path.exists(rd_folder_path):
            os.mkdir(rd_folder_path)
        compressed_dumps_list = os.listdir(wikipedia_dumps_folder)
        page_counter = 0
        for file_name in compressed_dumps_list:
            compressed_dump_file_path = os.path.join(wikipedia_dumps_folder, file_name)
            if not compressed_dump_file_path.endswith(".7z"):
                continue
            dump_file_name, _ = os.path.splitext(os.path.basename(compressed_dump_file_path))
            rdd_folder_path = os.path.join(rd_folder_path, dump_file_name)
            if not os.path.exists(rdd_folder_path):
                os.mkdir(rdd_folder_path)
            else:
                continue
            decompressed_dump_file_path = os.path.splitext(compressed_dump_file_path)[0]
            # TODO use only one of these
            with libarchive.public.file_reader(compressed_dump_file_path) as e:
                for entry in e:
                    with open(decompressed_dump_file_path, "wb") as fh:
                        for block in entry.get_blocks():
                            fh.write(block)
            # compressed_dump_file = io.open(compressed_dump_file_path, "rb")
            # compressed_dump = py7zlib.Archive7z(compressed_dump_file)
            # decompressed_dump_file = io.open(decompressed_dump_file_path, "w")
            # for f in compressed_dump.getmembers():
            #    data = f.read().decode("utf-8")
            #    decompressed_dump_file.write(data)
            # decompressed_dump_file.close()
            decompressed_dump_file = io.open(decompressed_dump_file_path, "r", encoding="utf-8")
            page_text = ""
            for i, line in enumerate(decompressed_dump_file):
                line = line.strip()
                if line == "<page>":
                    page_text = ""
                page_text += "\n" + line
                if line == "</page>":
                    revisions_list = []
                    page_tree = bs4.BeautifulSoup(page_text, "html.parser")
                    previous_text = ""
                    for revision_tag in page_tree.find_all("revision"):
                        revision_text = revision_tag.find_all("text")[0].text
                        if previous_text:
                            a = [t for t in self._wikitext_segmenter(previous_text) if t]
                            b = [t for t in self._wikitext_segmenter(revision_text) if t]
                            s = difflib.SequenceMatcher(None, a, b)
                            for tag, i1, i2, j1, j2 in s.get_opcodes():
                                if tag == "equal":
                                    continue
                                revisions_list.append({
                                    "old_value": a[i1:i2],
                                    "new_value": b[j1:j2],
                                    "left_context": a[i1 - self.REVISION_WINDOW_SIZE:i1],
                                    "right_context": a[i2:i2 + self.REVISION_WINDOW_SIZE]
                                })
                        previous_text = revision_text
                    if revisions_list:
                        page_counter += 1
                        if self.VERBOSE and page_counter % 100 == 0:
                            for entry in revisions_list:
                                print("----------Page Counter:---------\n", page_counter,
                                      "\n----------Old Value:---------\n", entry["old_value"],
                                      "\n----------New Value:---------\n", entry["new_value"],
                                      "\n----------Left Context:---------\n", entry["left_context"],
                                      "\n----------Right Context:---------\n", entry["right_context"],
                                      "\n==============================")
                        json.dump(revisions_list, open(os.path.join(rdd_folder_path, page_tree.id.text + ".json"), "w"))
            decompressed_dump_file.close()
            os.remove(decompressed_dump_file_path)

### Trash

In [ ]:
#First Method 
from bs4 import BeautifulSoup
import select
transport = client.get_transport()
channel = transport.open_session()
channel.exec_command("cat ./wikidump/enwiki-20190201-pages-meta-history27.xml-p53988643p54279382")
#while True:
    rl, wl, xl = select.select([channel],[],[],0.0)
    if len(rl) > 0:
        #print(channel.recv(1024))
        soup = BeautifulSoup(channel.recv(16384), 'html.parser')
        #print(soup.prettify())
        print(soup.find_all('page'))

# References
- https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python
- https://github.com/BigDaMa/baran/blob/master/baran.py
("_wikitext_segmenter" and "extract_revisions" methods)
